In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.settings import UC_MERCED_DATA_DIRECTORY, PATTERN_NET_DATA_DIRECTORY
from src.measures import anmrr
from sklearn.metrics.pairwise import euclidean_distances
from src.data.ucmerced_dataset import TripletDataModule
from src.utils import calculate_embeddings_torch
import torch
models = ['resnet18', 'resnet50', 'resnet101']
datasets = [
    # ("uc_merced", "UC Merced", UC_MERCED_DATA_DIRECTORY),
    ("pattern_net", "PatternNet", PATTERN_NET_DATA_DIRECTORY),
]

In [3]:
for dataset_path_name, dataset_name, dataset_path in datasets:
    for model in models:
        dm = TripletDataModule(dataset_path, 224, 0.8, 100)
        dm.setup(None)
        triplet_retriever = torch.hub.load('pytorch/vision', model, pretrained=True)
        triplet_retriever = triplet_retriever.cuda()
        paths, embeddings, classes = calculate_embeddings_torch(triplet_retriever, dm.val_dataloader())
        anmrr_value, anmrr_per_class, nmrr = anmrr(embeddings, classes, euclidean_distances, class_mean=True, all_queries=True)
        class_names = dm.class_names
        print(dataset_name, model, anmrr_value)

Using cache found in C:\Users\Szymon/.cache\torch\hub\pytorch_vision_master
61it [00:08,  7.08it/s]
Using cache found in C:\Users\Szymon/.cache\torch\hub\pytorch_vision_master
61it [00:18,  3.31it/s]
Using cache found in C:\Users\Szymon/.cache\torch\hub\pytorch_vision_master
61it [00:29,  2.10it/s]


PatternNet resnet18 0.5177311447265464
PatternNet resnet50 0.4871043397629931
PatternNet resnet101 0.4809735510100074


In [5]:
import torch
retriever = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)

Using cache found in C:\Users\Szymon/.cache\torch\hub\pytorch_vision_master
